In [121]:
%reset -f
%cd /home/assaf/jupyter/chess


import chess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle

def get_np_board(board_):
    matrix = np.zeros((8, 8), dtype=np.int8)
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 2,
        chess.BISHOP: 3,
        chess.ROOK: 4,
        chess.QUEEN: 5,
        chess.KING: 6,
    }
    for square, piece in board.piece_map().items():
        row = 7 - chess.square_rank(square)
        col = chess.square_file(square)
        value = piece_values[piece.piece_type]
        matrix[row, col] = value if piece.color == chess.WHITE else -value    
    return matrix

/home/assaf/jupyter/chess


In [ ]:

!rm -fr all_states_ds.pkl

df = pd.read_csv("games.csv")

try:
    with open("all_states_ds.pkl", "rb") as f:
        states_df = pickle.load(f) 
    print('loaded states and pickle from disk successfully')

except Exception:
    print('cannot load states from disk. calculating')
    states = []
    for i in range(len(df)):
        board = chess.Board()
        thisgame = df.loc[i]
        if thisgame['victory_status'] == 'outoftime':
            continue

        game_san = thisgame.moves.split()
        for j, step_san in enumerate(game_san):
            move = board.parse_san(step_san)  # Converts SAN to Move
            board.push(move)  # Make the move on the board

            state = []
            state.append(i)
            state.append(j+1)
            state.append(thisgame['turns']) 
            state.append(thisgame['victory_status'])  
            state.append(thisgame['winner'])
            state.append(thisgame['white_rating'])
            state.append(thisgame['black_rating'])
            state.append([board.has_queenside_castling_rights(chess.WHITE), board.has_kingside_castling_rights(chess.WHITE)])
            state.append([board.has_queenside_castling_rights(chess.BLACK), board.has_kingside_castling_rights(chess.BLACK)])   
            state.append(get_np_board(board))

            # state.append(str(board).replace(" ", "").replace("\n", "/"))
            # state.append(str(board).replace(" ", "").replace("\n", ""))
            states.append(state)

        # print (i, len(states))
    states_df = pd.DataFrame(states, columns=[ "gameid","turn", "turns", 'victory_status', 'winner', \
                                            'white_rating', 'black_rating',\
                                            'white_castling', 'black_castling', \
                                            'matrix', ])     
    with open("all_states_ds.pkl", "wb") as f:
        pickle.dump(states_df, f)
print (f'dataset length is {len(states_df)} ' )

ttsplit = 1080598
train = states_df.iloc[:ttsplit]
test = states_df.iloc[ttsplit:]
display(train.head())
test = test.reset_index(drop=True)
display(test.head())

with open("train_states_ds.pkl", "wb") as f:
    pickle.dump(train, f)
with open("test_states_ds.pkl", "wb") as f:
    pickle.dump(test, f)

cannot load states from disk. calculating
dataset length is 1090619 


,gameid,turn,turns,victory_status,winner,white_rating,black_rating,white_castling,black_castling,matrix
0,1,1,16,resign,black,1322,1261,"[True, True]","[True, True]","[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1..."
1,1,2,16,resign,black,1322,1261,"[True, True]","[True, True]","[[-4, 0, -3, -5, -6, -3, -2, -4], [-1, -1, -1,..."
2,1,3,16,resign,black,1322,1261,"[True, True]","[True, True]","[[-4, 0, -3, -5, -6, -3, -2, -4], [-1, -1, -1,..."
3,1,4,16,resign,black,1322,1261,"[True, True]","[True, True]","[[-4, 0, -3, -5, -6, -3, -2, -4], [-1, -1, -1,..."
4,1,5,16,resign,black,1322,1261,"[True, True]","[True, True]","[[-4, 0, -3, -5, -6, -3, -2, -4], [-1, -1, -1,..."


,gameid,turn,turns,victory_status,winner,white_rating,black_rating,white_castling,black_castling,matrix
0,19908,1,60,resign,white,1563,1582,"[True, True]","[True, True]","[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1..."
1,19908,2,60,resign,white,1563,1582,"[True, True]","[True, True]","[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1..."
2,19908,3,60,resign,white,1563,1582,"[True, True]","[True, True]","[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1..."
3,19908,4,60,resign,white,1563,1582,"[True, True]","[True, True]","[[-4, -2, -3, -5, -6, -3, 0, -4], [-1, -1, -1,..."
4,19908,5,60,resign,white,1563,1582,"[True, True]","[True, True]","[[-4, -2, -3, -5, -6, -3, 0, -4], [-1, -1, -1,..."
